In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

import os
from dotenv import load_dotenv

In [32]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

In [5]:
examples = [
    {
        "country": "What do you konw about France?",
        "answer": """
        Here is what I Know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Curo
        """,
    }, {
        "country": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    }, {
        "country": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """
    }
]

In [41]:
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(examples)]

In [42]:
example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")


In [43]:
example_selector = RandomExampleSelector(
    examples=examples,
)

In [44]:
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"], 
)

In [45]:
prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI:\n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [14]:
chain = prompt | chat

In [15]:
chain.invoke({"country": "Turkey"})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebab and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebab and Baklava\n        Currency: Turkish Lira')